In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Load CSV data into Pandas DataFrame
df = pd.read_csv("Demand.csv", sep=';')

# Get basic information about data
print(df.info())

# Split the data based on the 'Day' column
train_data = df[df['Day'] <= 250] # Days 1 to 250 for training
test_data = df[df['Day'] > 250] # Days 251 to 300 for testing

# Separate features (X) and target (y)
X_train = train_data[['Day']]
y_train = train_data['Demand']

X_test = test_data[['Day']]
y_test = test_data['Demand']

# Train the linear regression model on the training dataset
linreg_model = LinearRegression()
linreg_model.fit(X_train, y_train)

# Predict on train and test datasets
linreg_model_prediction_train = linreg_model.predict(X_train)
linreg_model_prediction_test = linreg_model.predict(X_test)

# Evaluate and display model accuracy with Mean Absolute Error for both train and test datasets
print(f"Mean absolute error in train dataset using Linear Regression Model: {mean_absolute_error(y_train, linreg_model_prediction_train):.2f}")
print(f"Mean absolute error in test dataset using Linear Regression Model: {mean_absolute_error(y_test, linreg_model_prediction_test):.2f}")

# Evaluate and display model accuracy with R2 Score for both train and test datasets
print(f"\nR2 score in train dataset using Linear Regression Model: {r2_score(y_train, linreg_model_prediction_train):.2f}")
print(f"R2 score in test dataset using Linear Regression Model: {r2_score(y_test, linreg_model_prediction_test):.2f}")


# Train the non-linear Neural Network model on the training dataset
nn_model = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(30, activation='tanh',),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(30, activation='relu',),
    tf.keras.layers.Dense(1)
])

nn_model.compile(
    tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

nn_model.fit(X_train, y_train, validation_data=(X_test, y_test) , epochs=50, batch_size=10)

# Predict on train and test datasets
nn_model_prediction_train = nn_model.predict(X_train)
nn_model_prediction_test = nn_model.predict(X_test)

# Evaluate and display model accuracy with Mean Absolute Error for both train and test datasets
print(f"Mean absolute error in train dataset using Linear Regression Model: {mean_absolute_error(y_train, nn_model_prediction_train):.2f}")
print(f"Mean absolute error in test dataset using Linear Regression Model: {mean_absolute_error(y_test, nn_model_prediction_test):.2f}")

# Evaluate and display model accuracy with R2 Score for both train and test datasets
print(f"\nR2 score in train dataset using Linear Regression Model: {r2_score(y_train, nn_model_prediction_train):.2f}")
print(f"R2 score in test dataset using Linear Regression Model: {r2_score(y_test, nn_model_prediction_test):.2f}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Day     300 non-null    int64
 1   Demand  300 non-null    int64
dtypes: int64(2)
memory usage: 4.8 KB
None
Mean absolute error in train dataset using Linear Regression Model: 446.38
Mean absolute error in test dataset using Linear Regression Model: 578.07

R2 score in train dataset using Linear Regression Model: 0.70
R2 score in test dataset using Linear Regression Model: -1.25
Epoch 1/50


/Users/dineshbisht/masterdegree/myproject_env/lib/python3.13/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 12552623.0000 - mae: 3392.9314 - val_loss: 21492440.0000 - val_mae: 4610.8398
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 992us/step - loss: 12537593.0000 - mae: 3390.7615 - val_loss: 21467286.0000 - val_mae: 4608.1118
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 897us/step - loss: 12518510.0000 - mae: 3387.9614 - val_loss: 21429838.0000 - val_mae: 4604.0474
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 918us/step - loss: 12490324.0000 - mae: 3383.8113 - val_loss: 21378622.0000 - val_mae: 4598.4814
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 809us/step - loss: 12446730.0000 - mae: 3377.4177 - val_loss: 21311856.0000 - val_mae: 4591.2163
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 817us/step - loss: 12392782.0000 - mae: 3369.4465 - val_loss: 21229640.0000 - val_mae: 4582.2539
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 868us/step - loss: 12326632.0000 - mae: 3359.5442 - val_loss: 21130626.0000 - val_mae: 4571.4370
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/